In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [15]:
# Load Dataset
df = pd.read_csv("/content/Alphabets_data.csv")

In [16]:
# Data Exploration
print("Dataset Shape:", df.shape)
print("Class Distribution:\n", df['letter'].value_counts())

Dataset Shape: (20000, 17)
Class Distribution:
 letter
U    813
D    805
P    803
T    796
M    792
A    789
X    787
Y    786
N    783
Q    783
F    775
G    773
E    768
B    766
V    764
L    761
R    758
I    755
O    753
W    752
S    748
J    747
K    739
C    736
H    734
Z    734
Name: count, dtype: int64


In [17]:
# Handling Missing Values
df.dropna(inplace=True)

In [18]:
# Separating Features and Labels
X = df.drop(columns=['letter'])
y = df['letter']

In [19]:
# Encoding Labels
le = LabelEncoder()
y = le.fit_transform(y)

In [20]:
# Normalizing Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [21]:
# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Function to Create ANN Model
def create_model(hidden_layers=1, neurons=64, activation='relu', learning_rate=0.001):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    for _ in range(hidden_layers):
        model.add(keras.layers.Dense(neurons, activation=activation))

    model.add(keras.layers.Dense(len(np.unique(y)), activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [23]:
# Training a Simple Model
model = create_model()
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_data=(X_test, y_test))

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3170 - loss: 2.5522 - val_accuracy: 0.7032 - val_loss: 1.2304
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7184 - loss: 1.1130 - val_accuracy: 0.7747 - val_loss: 0.8849
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7795 - loss: 0.8347 - val_accuracy: 0.8070 - val_loss: 0.7382
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8019 - loss: 0.7172 - val_accuracy: 0.8240 - val_loss: 0.6487
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8253 - loss: 0.6287 - val_accuracy: 0.8363 - val_loss: 0.5861
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8398 - loss: 0.5694 - val_accuracy: 0.8512 - val_loss: 0.5355
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8559 - loss: 0.5095 - val_accuracy: 0.8568 - val_loss: 0.4960
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8654 - loss: 0.4692 - val_accuracy: 0.

In [24]:
# Evaluating the Model
y_pred = np.argmax(model.predict(X_test), axis=1)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.91175
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.94      0.95       149
           1       0.88      0.89      0.89       153
           2       0.93      0.93      0.93       137
           3       0.84      0.94      0.89       156
           4       0.91      0.91      0.91       141
           5       0.90      0.86      0.88       140
           6       0.88      0.88      0.88       160
           7       0.79      0.74      0.76       144
           8       0.95      0.89      0.92       146
           9       0.95      0.93      0.94       149
          10       0.83      0.82      0.82       130
          11       0.99      0.95      0.97       155
          12       0.96      0.96      0.96       168
          13       0.93      0.87      0.90       151
          14       0.88      0.92      0.90       145
          15       0.96      0.91      0.94       17

In [25]:
# Basic Hyperparameter Tuning
best_accuracy = 0
best_params = {}
for layers in [1, 2]:
    for neurons in [32, 64]:
        for lr in [0.001, 0.01]:
            model = create_model(hidden_layers=layers, neurons=neurons, learning_rate=lr)
            model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
            acc = model.evaluate(X_test, y_test, verbose=0)[1]
            if acc > best_accuracy:
                best_accuracy = acc
                best_params = {'layers': layers, 'neurons': neurons, 'lr': lr}

In [26]:
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

Best Hyperparameters: {'layers': 2, 'neurons': 64, 'lr': 0.01}
Best Accuracy: 0.9212499856948853
